# Лабораторна робота № 4

## ІП-14 Бабіч Денис (09.07.2003)

---

# Підготовчий етап

## Імпортування необхідних модулів

In [133]:
import cv2
import numpy as np

## Створення необхідних функцій

In [420]:
def preprocess_frame(frame: np.ndarray, blur_strength: int, threshold: float, kernel_size: int, *roi_polygons: list) -> np.ndarray: 
    preprocessed_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    preprocessed_frame = cv2.equalizeHist(preprocessed_frame)
    
    preprocessed_frame = cv2.GaussianBlur(preprocessed_frame, (blur_strength, blur_strength), 0)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    preprocessed_frame = cv2.morphologyEx(preprocessed_frame, cv2.MORPH_CLOSE, kernel)
    preprocessed_frame = cv2.erode(preprocessed_frame, kernel, iterations = 1)
    preprocessed_frame = cv2.dilate(preprocessed_frame, kernel, iterations = 1)
    
    preprocessed_frame = cv2.Canny(preprocessed_frame, threshold / 2, threshold)

    mask = np.zeros_like(preprocessed_frame)

    for roi_polygon_vertices in roi_polygons:
        cv2.fillPoly(mask, [np.array(roi_polygon_vertices, dtype = np.int32)], 255)

    preprocessed_frame = cv2.bitwise_and(preprocessed_frame, mask)

    return preprocessed_frame

def determine_obejcts_rectangles(frame: np.ndarray, width_min: int, width_max: int, height_min: int, height_max: int) -> list:
    objects_rectangles = []

    contours, _ = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, width, height = cv2.boundingRect(contour)

        if (width_min < width < width_max) and (height_min < height < height_max):
            objects_rectangles.append((x, y, width, height))

    return objects_rectangles

def overlay_objects_rectangles(frame: np.ndarray, rectangles: list, roi_title: str = "", color: tuple = (0, 255, 0), thickness: int = 2) -> np.ndarray:
    ROI_TITLE_OFFSET = 10

    FONT_SCALE = 0.5
    FONT_LINE_TYPE = 2

    for (x, y, width, height) in rectangles:
        cv2.rectangle(frame, (x, y), (x + width, y + height), color, thickness)

        if roi_title:
            cv2.putText(frame, roi_title, (x, y - ROI_TITLE_OFFSET), cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, color, FONT_LINE_TYPE)

    return frame

---

# Основний етап

## Застосування на прикладі відео

In [425]:
KEY_ESCAPE = 27

CAPTURE = cv2.VideoCapture("video.mp4")
FPS = int(CAPTURE.get(cv2.CAP_PROP_FPS))

THRESHOLD = 300
KERNEL_SIZE = 5
BLUR_STRENGTH = 5

WIDTH_MIN = 40
WIDTH_MAX = 110
HEIGHT_MIN = 30
HEIGHT_MAX = 70

FRAME_HEIGHT, FRAME_WIDTH = CAPTURE.read()[1].shape[:2]

FRAME_HALF_HEIGHT = FRAME_HEIGHT / 2

OFFSET_MARKUP = 50
OFFSET_BOUNDS = 75

ROAD_WIDTH = 300

ROI_TOP_ROAD = [(0, FRAME_HEIGHT / 2),
                (0, FRAME_HEIGHT / 2),
                (FRAME_WIDTH, FRAME_HALF_HEIGHT - OFFSET_MARKUP),
                (FRAME_WIDTH, FRAME_HEIGHT - OFFSET_MARKUP)]

ROI_BOTTOM_ROAD = [()]

ROI_TITLE = "Car"

while CAPTURE.isOpened():
    ret, frame = CAPTURE.read()

    if not ret:
        break

    preprocessed_frame = preprocess_frame(frame, BLUR_STRENGTH, THRESHOLD, KERNEL_SIZE, ROI_TOP_ROAD)
    objects_roi = determine_obejcts_rectangles(preprocessed_frame, WIDTH_MIN, WIDTH_MAX, HEIGHT_MIN, HEIGHT_MAX)

    processed_frame = overlay_objects_rectangles(frame, objects_roi, ROI_TITLE)

    cv2.imshow("", preprocessed_frame)

    if cv2.waitKey(FPS) & 0xFF == KEY_ESCAPE:
        break

CAPTURE.release()
cv2.destroyAllWindows()